[texto del enlace](https://)# Batalla de los vecinos


## INTRODUCCION

Este proyecto es para quienes planean abrir una cafetería en Ankara, Turquía. Este proyecto sugiere las mejores ubicaciones para Coffee Houses en Ankara. Ankara es la capital de Turquía con una población de 5M. La cultura turca tiene profundas raíces en el café. Además, se está volviendo popular debido a sus precios asequibles. El consumo de café ha aumentado un 13% y el consumo medio es de 1,1 kg/persona en 2018.
Este informe explora qué barrios de Ankara tienen más y las mejores cafeterías. Además, este proyecto responde a las preguntas "¿Dónde debo abrir una cafetería?" y “¿Dónde debo quedarme si quiero un rico café?”

## Data

Distrito de Ankara se obtienen de https://en.wikipedia.org/wiki/Ankara_Province
Los valores de latitud y longitud se obtienen utilizando "geocodificador".
Todos los datos relacionados con las ubicaciones se obtendrán mediante la API de FourSquare y las bibliotecas de Python.

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
!conda install -c conda-forge geopy --yes
import geocoder

Solving environment: ...working... 
  - anaconda/win-64::conda-4.8.1-py36_0
  - defaults/win-64::conda-4.8.1-py36_0done

# All requested packages already installed.



In [ ]:
wiki_link = 'https://en.wikipedia.org/wiki/Ankara_Province'
wiki_page = requests.get(wiki_link, headers = headers)
wiki_page

<Response [200]>

In [ ]:
soup = BeautifulSoup(wiki_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [ ]:
rows = table.find_all('tr')

In [ ]:
columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
columns

['District', 'Population (2017)', 'Area (km²)', 'Density (per km²)']

In [ ]:
df_ankara = pd.DataFrame(columns = columns)

In [ ]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df_ankara = df_ankara.append(pd.Series(values, index = columns), ignore_index = True)

        df_ankara

In [ ]:
df_ankara.head()

,District,Population (2017),Area (km²),Density (per km²)
0,Akyurt,32.863,369,89
1,Altındağ,371.366,123,3.019
2,Ayaş,12.289,1.041,12
3,Bala,21.682,1.851,12
4,Beypazarı,48.476,1.697,29


In [ ]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ankara, Turkey'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [ ]:
districts = df_ankara['District']    
coordinates = [get_latlng(districts) for districts in districts.tolist()]

In [ ]:
df_ankara_loc = df_ankara

df_ankara_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_ankara_loc['Latitude'] = df_ankara_coordinates['Latitude']
df_ankara_loc['Longitude'] = df_ankara_coordinates['Longitude']

In [ ]:
df_ankara_loc.head()

,District,Population (2017),Area (km²),Density (per km²),Latitude,Longitude
0,Akyurt,32.863,369,89,40.13082,33.08719
1,Altındağ,371.366,123,3.019,39.94171,32.85445
2,Ayaş,12.289,1.041,12,40.01516,32.33270
3,Bala,21.682,1.851,12,39.55391,33.12352
4,Beypazarı,48.476,1.697,29,40.16811,31.92052
